In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
tweets = pd.read_csv("D:/Data Science/Practice/Twitter US Airline Sentiment Analysis/tweets.csv") #10980 rows 12 cols

In [3]:
tweets.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
print(tweets['negativereason_gold'].nunique())
print(tweets['negativereason_gold'].value_counts(),"\n")

11
Customer Service Issue                      9
Late Flight                                 3
Cancelled Flight                            3
Can't Tell                                  2
Late Flight\nFlight Attendant Complaints    1
Cancelled Flight\nCustomer Service Issue    1
Late Flight\nCancelled Flight               1
Customer Service Issue\nLost Luggage        1
Customer Service Issue\nCan't Tell          1
Bad Flight                                  1
Lost Luggage\nDamaged Luggage               1
Name: negativereason_gold, dtype: int64 



## Cleaning

In [5]:
drop_cols = ['airline_sentiment_gold','name','tweet_id', 'retweet_count','tweet_created','user_timezone','tweet_coord','tweet_location']
tweets.drop(drop_cols, axis = 1, inplace=True)

In [6]:
tweets.head(3)

,airline_sentiment,airline,negativereason_gold,text
0,negative,Southwest,NaN,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,Southwest,NaN,@SouthwestAir seeing your workers time in and ...
2,positive,United,NaN,@united Flew ORD to Miami and back and had gr...


In [7]:
# Stop Words
stops = stopwords.words('english')
stops += list(punctuation)
stops += ['flight','airline','flights','AA']

In [8]:
abbreviations = {'ppl': 'people','cust':'customer','serv':'service','mins':'minutes','hrs':'hours','svc': 'service',
           'u':'you','pls':'please'}

tweet_index = tweets[~tweets.negativereason_gold.isna()].index

for index, row in tweets.iterrows():
    tweet = row.text
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace '#abc' with abc
    tweet = tweet.strip('\'"') #trim tweet
    words = []
    for word in tweet.split():
#         if not has Numbers(word):
        if word.lower() not in stops:
            if word in list(abbreviations.keys()):
                words.append(abbreviations[word])
            else:
                words.append(word.lower())   
    tweet = " ".join(words)
    tweet = " %s %s" % (tweet, row.airline)
    row.text = tweet
    if index in tweet_index:
        row.text = " %s %s" % (row.text, row.negativereason_gold)

del tweets['negativereason_gold']

In [9]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

for index, row in tweets.iterrows():
    row.text = deEmojify(row.text)

In [10]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

for index, row in tweets.iterrows():
    words = row.text.split()
    new_words = []
    for word in words:
        if not hasNumbers(word):
            new_words.append(word)
    row.text = " ".join(new_words)

In [11]:
def convert_Sentiment(sentiment):
    if  sentiment == "positive":
        return 2
    elif sentiment == "neutral":
        return 1
    elif sentiment == "negative":
        return 0

tweets.airline_sentiment = tweets.airline_sentiment.apply(lambda x : convert_Sentiment(x))

In [12]:
tweets.head()

,airline_sentiment,airline,text
0,0,Southwest,"scheduled morning, days fact, yes..not sure ev..."
1,2,Southwest,seeing workers time time going beyond love fly...
2,2,United,"flew ord miami back great crew, service legs. ..."
3,0,Southwest,that's horse radish Southwest
4,0,United,"ord delayed air force one, last sbn minutes la..."


## Creating vocab and data formatting

In [13]:
v = TfidfVectorizer(analyzer='word', max_features=3150, max_df = 0.8, ngram_range=(1,1))
x= v.fit_transform(tweets.text)
y= tweets['airline_sentiment']

In [14]:
X_train , X_test , y_train , y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [16]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred1 = clf.predict(X_test)
accuracy_score(pred1, y_test)

C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7663934426229508

In [17]:
clf = SVC()
clf.fit(X_train, y_train)
pred2 = clf.predict(X_test)
accuracy_score(pred2, y_test)

0.7654826958105647

In [18]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred3 = clf.predict(X_test)
accuracy_score(pred3, y_test)

0.755464480874317

In [19]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred4 = clf.predict(X_test)
accuracy_score(pred4, y_test)

0.6602914389799636

In [20]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
pred5 = clf.predict(X_test)
accuracy_score(pred5, y_test)

0.7427140255009107

In [21]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
pred6 = clf.predict(X_test)
accuracy_score(pred6, y_test)

0.7071948998178507

In [22]:
clf = XGBClassifier()
clf.fit(X_train, y_train)
pred7 = clf.predict(X_test)
accuracy_score(pred7, y_test)

0.7504553734061931

In [24]:
cr = classification_report(y_test, pred1)
print("Classification Report:\n----------------------\n", cr)

Classification Report:
----------------------
               precision    recall  f1-score   support

           0       0.87      0.84      0.85      1356
           1       0.55      0.64      0.59       458
           2       0.70      0.67      0.68       382

    accuracy                           0.77      2196
   macro avg       0.71      0.71      0.71      2196
weighted avg       0.78      0.77      0.77      2196

